In [1]:
import logging
import os
import numpy as np
import pandas as pd
import time

from data_loader import DataLoader

In [3]:
if __name__ == '__main__':

    path = 'C:/Users/Akis/OneDrive/Masters/Web Economics/Data/'

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    logging.info('starting train_via_rf.py')

    target_col_name = 'click'
    data_filename = 'validation.csv'

    logging.info('loading DF ' + data_filename)
    train_dl = DataLoader()
    train_dl.load_file(path, data_filename)
    df = train_dl.get_df_copy()

2018-03-03 23:25:09,566 : INFO : starting train_via_rf.py
2018-03-03 23:25:09,568 : INFO : loading DF validation.csv
2018-03-03 23:25:09,568 : INFO : Loading C:/Users/Akis/OneDrive/Masters/Web Economics/Data/validation.csv
2018-03-03 23:25:11,289 : INFO : file  loaded


In [4]:
class Scorer(object):

    def second_largest(self,numbers):
        count = 0
        topBid = secondBid = -1.0
        for x in numbers:
            count += 1
            if x > secondBid:
                if x >= topBid:
                    topBid, secondBid = x, topBid
                else:
                    secondBid = x
        return secondBid if count >= 2 else None
    
    def Formula1CombineCTRBaseBid(self,CTR,BaseBid,TotalAuctionCount,CurrentAuctionNumber):
        #CTR=(((CTR+0.5)**2)-0.5) #increases CTRs above 0.5, decreases CTRs below 0.5
        #bids = (CTR/0.5)*BaseBid #formula given
        bids = ((CTR/0.5)*BaseBid)*(1+CurrentAuctionNumber*10/TotalAuctionCount) #bids increase with number of auctions participated in
        return bids
    def Formula2CombineCTRBaseBid(self,CTR,BaseBid,TotalAuctionCount,CurrentAuctionNumber):
        #CTR=(((CTR+0.5)**2)-0.5) #increases CTRs above 0.5, decreases CTRs below 0.5
        bids = (CTR/0.5)*BaseBid #formula given
        #bids = ((CTR/0.5)*BaseBid)*(1+CurrentAuctionNumber*10/TotalAuctionCount) #bids increase with number of auctions participated in
        return bids

    def set_df(self,clicksDF,predictionsBaseBid,predictionsCTR,column_names, budget, reserve):
        

        CTRdf=clicksDF.copy(True)
        BaseBiddf=clicksDF.copy(True)
        for i in range(0,len(predictionsCTR),1):
            name=column_names[i]
            CTRdf[name] = predictionsCTR[i]
            BaseBiddf[name] = predictionsBaseBid[i]
        
        #df = df.copy(True)
        balances = np.ones(len(column_names)) * budget
        df_teams_CTR = CTRdf[column_names]
        df_teams_BaseBid = BaseBiddf[column_names]
        BidsDF=df_teams_CTR
        #print(BidsDF)
        CTRdf['winner'] = -1
        CTRdf['price'] = 0
        logging.info('calculating price paid (second highest). This takes some time...')
           
        for i in range(len(CTRdf)):
            #prices = (df_teams_CTR.loc[i].values/0.5)*df_teams_BaseBid.loc[i].values
            prices = self.Formula1CombineCTRBaseBid(df_teams_CTR.loc[i].values,df_teams_BaseBid.loc[i].values,len(CTRdf),i)
            #prices2 = self.Formula2CombineCTRBaseBid(df_teams_CTR.loc[i].values,df_teams_BaseBid.loc[i].values,len(CTRdf),i)
            #prices = np.append(prices1,prices2)
            #print(prices)
            #BidsDF[i]=prices
            valid1 = balances > prices
            valid2 = prices > 0.0
            valid3 = valid1 * valid2
            new_prices = valid3 * prices
            winner = np.argmax(new_prices)
            if (new_prices[winner]>0):
                price = self.second_largest(new_prices)
                price = np.max([reserve,price])
                CTRdf.loc[i, 'price'] = price
                CTRdf.loc[i, 'winner'] = winner
                balances[winner] -= price

            if i % int(len(CTRdf)/49) == 0:
                logging.info('%.2f complete',(float(i)/len(CTRdf)))

            if np.sum(balances > 0.0) == 0: # if no balances are positive, we don't need to process the rest of the data frame
                break

        logging.info('calculating wins by team')
        wins_by_team = CTRdf.winner.value_counts()
        teams = CTRdf.winner.unique()
        print(BidsDF)
        team_names = []
        team_wins = []
        team_clicks = []
        for team_index in teams:
            if team_index >= 0:
                temp_df = CTRdf.loc[CTRdf['winner'] == team_index]
                clicks = np.sum(temp_df['click'].values)
                team_name = df_teams_CTR.columns[team_index]
                team_names.append(team_name)
                team_wins.append(wins_by_team[team_index])
                team_clicks.append(clicks)

                logging.info(
                    'team ' + team_name + ' ' + str(team_index) + ' wins:' + str(wins_by_team[team_index]) + ' clicks:' + str(clicks))
        df_summary = pd.DataFrame({"team_name": team_names, "win": team_wins, "click": team_clicks})
        return(df_summary,CTRdf)

In [5]:
clicksFile = df[['click']].copy(True)
Random1BaseBid = np.random.randint(227, 230, clicksFile.shape[0])
Constant1BaseBid = np.ones(clicksFile.shape[0]) * 278
Random2BaseBid = np.random.randint(297, 301, clicksFile.shape[0])
Random1CTR = np.random.randint(0, 100, clicksFile.shape[0])/100
Constant1CTR = np.random.randint(0, 100, clicksFile.shape[0])/100
Random2CTR = np.random.randint(0, 100, clicksFile.shape[0])/100
print(Constant1BaseBid)
print(Constant1CTR)
#print(clicksFile.head())


[278. 278. 278. ... 278. 278. 278.]
[0.24 0.27 0.48 ... 0.52 0.56 0.3 ]


In [6]:
#for i in range(1,300,10):
#    print((Random2CTR[i]/0.5)*Random2BaseBid[i])

In [7]:
start = time.time()
s = Scorer()
df_summary, df_full = s.set_df(clicksFile,[Random1BaseBid,Random2BaseBid,Constant1BaseBid],[Random1CTR,Random2CTR,Constant1CTR],['Random1CTR','Random2CTR','Constant1CTR'],6250, 227)

end = time.time()
print("Time:",end - start,"s")

print(df_summary.head())

2018-03-03 23:25:21,761 : INFO : calculating price paid (second highest). This takes some time...
2018-03-03 23:25:21,812 : INFO : 0.00 complete
2018-03-03 23:25:22,186 : INFO : calculating wins by team
2018-03-03 23:25:22,199 : INFO : team Random2CTR 1 wins:22 clicks:0
2018-03-03 23:25:22,201 : INFO : team Random1CTR 0 wins:25 clicks:0
2018-03-03 23:25:22,203 : INFO : team Constant1CTR 2 wins:20 clicks:0


        Random1CTR  Random2CTR  Constant1CTR
0             0.93        0.88          0.24
1             0.70        0.48          0.27
2             0.86        0.40          0.48
3             0.75        0.92          0.79
4             0.50        0.20          0.70
5             0.31        0.14          0.22
6             0.13        0.64          0.71
7             0.08        0.22          0.26
8             0.61        0.12          0.85
9             0.21        0.03          0.79
10            0.86        0.09          0.84
11            0.51        0.94          0.76
12            0.45        0.76          0.97
13            0.87        0.78          0.29
14            0.96        0.94          0.51
15            0.05        0.39          0.13
16            0.88        0.48          0.81
17            0.95        0.90          0.33
18            0.78        0.45          0.46
19            0.31        0.97          0.09
20            0.80        0.65          0.96
21        

In [ ]:
#print(df_full)